In [12]:
import os
import matplotlib.table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils_experimentation import SolverFormatInfo, Convergence_Experiment
from exp_spec_info import *
from bokeh.palettes import Category20
cat20 = Category20[20]

In [13]:
processed_data_population_pkl_path = "C:\\Users\\dosre\\dev\\thesis-data\\new-transfer\\processed_data_population.pkl"
output_plot_root_dir_path = "C:\\Users\\dosre\\dev\\thesis-data\\new-transfer\\experimentation_analysis"
inner_iter_plot_dir_path = os.path.join(output_plot_root_dir_path, "inner_iter")
outer_iter_plot_dir_path = os.path.join(output_plot_root_dir_path, "outer_iter")
os.makedirs(output_plot_root_dir_path, exist_ok=True)
os.makedirs(inner_iter_plot_dir_path, exist_ok=True)
os.makedirs(outer_iter_plot_dir_path, exist_ok=True)

In [14]:
processed_data_population = pd.read_pickle(processed_data_population_pkl_path)

In [15]:
def filter_setup_matrix_inner_iter(data, setup, matrix, inner_iter):
    return data[
        (data["setup"] == setup) &
        (data["matrix"] == matrix) &
        (data["inner_iter"] == inner_iter)
    ]

In [17]:
processed_data_population

#### 6.1 Unpreconditioned Sparse Summary Data

In [19]:
for matrix in [ALL_MATRICES[0]]:
    data = filter_setup_matrix_inner_iter(
        processed_data_population,
        "unpreconditioned",
        matrix,
        30
    )
    display(data)

#### Raw Convergence Data

In [8]:
matrices = processed_data_population["matrix"].unique()
for matrix in matrices:

    matrix_plot_dir_path = os.path.join(inner_iter_plot_dir_path, matrix)
    os.makedirs(matrix_plot_dir_path, exist_ok=True)

    matrix_data_population = processed_data_population.loc[processed_data_population["matrix"] == matrix]
    setups = matrix_data_population["setup"].unique()

    for setup in setups:

        setup_plot_dir_path = os.path.join(matrix_plot_dir_path, setup)
        os.makedirs(setup_plot_dir_path, exist_ok=True)
        
        matrix_setup_data_population = matrix_data_population.loc[matrix_data_population["setup"] == setup]
        inner_iters = matrix_setup_data_population["inner_iter"].unique()

        for inner_iter in inner_iters:

            convergence_data_to_plot = matrix_setup_data_population.loc[matrix_setup_data_population["inner_iter"] == inner_iter]
            plot_file_path = os.path.join(
                setup_plot_dir_path,
                f"inner_iter_{inner_iter:03d}.jpg"
            )

            conv_experiments = []
            for row in convergence_data_to_plot.iterrows():
                row = row[1]
                conv_experiments.append(Convergence_Experiment(row.solver, row.file_paths))

            fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
            ax1, ax2, ax3 = axs
            
            table_data = [["solver", "inner iter", "rel time", "rel res", "converged"]]
            
            fp64_base_elapsed_time_ms = 0.
            for exp_data in conv_experiments:
                if exp_data.id == "FP64":
                    fp64_base_elapsed_time_ms = exp_data.median_elapsed_time_ms
            
            for exp_data in conv_experiments:
                if exp_data.id in "FP64":
                    for ax in axs:
                        exp_data.plot_convergence_data(ax)
                elif exp_data.id in SolverFormatInfo.fp_ids:
                    exp_data.plot_convergence_data(ax1)
                elif exp_data.id in SolverFormatInfo.sd_vp_ids:
                    exp_data.plot_convergence_data(ax2)
                else:
                    exp_data.plot_convergence_data(ax3)
            
                relative_time = exp_data.median_elapsed_time_ms/fp64_base_elapsed_time_ms
                table_data.append(
                    [
                        exp_data.id,
                        int(exp_data.median_inner_iteration_count),
                        np.nan if np.isnan(exp_data.median_final_relres) else f"{relative_time:.2g}",
                        f"{exp_data.median_final_relres:.3g}",
                        exp_data.all_converged
                    ]
                )

            table = matplotlib.table.table(ax, cellText=table_data, loc="right", colWidths=[0.3, 0.1, 0.1, 0.1, 0.1])
            table.auto_set_font_size(False)
            table.set_fontsize(8)
            
            for ax in axs:
                ax.legend()
                ax.semilogy()
                ax.grid()
                ax.set_ylabel("$\\frac{|| b-Ax_{i}||_{2}}{||b-Ax_{0}||_{2}}$")
            ax3.set_xlabel("Inner Iteration")
            
            fig.suptitle(f"{matrix} {setup} GMRES({inner_iter})")
            
            plt.savefig(plot_file_path, bbox_inches="tight")

            plt.close()
            

In [9]:
matrices = processed_data_population["matrix"].unique()
for matrix in matrices:

    matrix_plot_dir_path = os.path.join(outer_iter_plot_dir_path, matrix)
    os.makedirs(matrix_plot_dir_path, exist_ok=True)

    matrix_data_population = processed_data_population.loc[processed_data_population["matrix"] == matrix]
    setups = matrix_data_population["setup"].unique()

    for setup in setups:

        setup_plot_dir_path = os.path.join(matrix_plot_dir_path, setup)
        os.makedirs(setup_plot_dir_path, exist_ok=True)
        
        matrix_setup_data_population = matrix_data_population.loc[matrix_data_population["setup"] == setup]
        inner_iters = matrix_setup_data_population["inner_iter"].unique()

        for inner_iter in inner_iters:

            convergence_data_to_plot = matrix_setup_data_population.loc[matrix_setup_data_population["inner_iter"] == inner_iter]
            plot_file_path = os.path.join(
                setup_plot_dir_path,
                f"inner_iter_{inner_iter:03d}.jpg"
            )

            conv_experiments = []
            for row in convergence_data_to_plot.iterrows():
                row = row[1]
                conv_experiments.append(Convergence_Experiment(row.solver, row.file_paths))

            fig, axs = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
            ax1, ax2, ax3 = axs
            
            table_data = [["solver", "outer iter", "rel time", "rel res", "converged"]]
            
            fp64_base_elapsed_time_ms = 0.
            for exp_data in conv_experiments:
                if exp_data.id == "FP64":
                    fp64_base_elapsed_time_ms = exp_data.median_elapsed_time_ms
            
            for exp_data in conv_experiments:
                if exp_data.id in "FP64":
                    for ax in axs:
                        exp_data.plot_outer_convergence_data(ax)
                elif exp_data.id in SolverFormatInfo.fp_ids:
                    exp_data.plot_outer_convergence_data(ax1)
                elif exp_data.id in SolverFormatInfo.sd_vp_ids:
                    exp_data.plot_outer_convergence_data(ax2)
                else:
                    exp_data.plot_outer_convergence_data(ax3)
            
                relative_time = exp_data.median_elapsed_time_ms/fp64_base_elapsed_time_ms
                table_data.append(
                    [
                        exp_data.id,
                        int(exp_data.median_outer_iteration_count),
                        np.nan if np.isnan(exp_data.median_final_relres) else f"{relative_time:.2g}",
                        f"{exp_data.median_final_relres:.3g}",
                        exp_data.all_converged
                    ]
                )

            table = matplotlib.table.table(ax, cellText=table_data, loc="right", colWidths=[0.3, 0.1, 0.1, 0.1, 0.1])
            table.auto_set_font_size(False)
            table.set_fontsize(8)
            
            for ax in axs:
                ax.legend()
                ax.semilogy()
                ax.grid()
                ax.set_ylabel("$\\frac{|| b-Ax_{i}||_{2}}{||b-Ax_{0}||_{2}}$")
            ax3.set_xlabel("Outer Iteration")
            
            fig.suptitle(f"{matrix} {setup} GMRES({inner_iter})")
            
            plt.savefig(plot_file_path, bbox_inches="tight")

            plt.close()
            